## Imports

In [4]:
from PIL import Image, ImageSequence
import os
import tqdm
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Extracting Images from TIF Files


### Extracting From A Single TIF File

In [2]:
def get_correct_num(num):
    if isinstance(num, int):
        num = str(num)
    if len(num) == 1:
        num = '00' + num
    elif len(num) == 2:
        num = '0' + num
    return num

In [3]:
def extract_frames(tif_path, save_path=None, raise_error=False, verbose=False):
    """
    Extracts frames from a tiff file

    Parameters
    ----------
    tif_path : str
        Path to the tiff file
    save_path : str
        Path to save the frames. If None, `save_path` will be defined using
        the tiff file name.
    raise_error : bool
        If True, will raise an error when the enumeration is ended else
        will just stop the enumeration.
    verbose : bool
        If True, will print the progress of the enumeration.

    Returns
    -------
    save_path : str
        Path to the folder where the frames were saved.
    """
    tif_file = Image.open(tif_path)
    if save_path is None:
        save_path = tif_path.replace('.tif', '').split(os.path.sep)[-1]
        save_path = os.path.join("extracted_images", save_path)
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    if verbose:
        print("Extracting to ", save_path)
    try:
        for i, page in enumerate(ImageSequence.Iterator(tif_file)):
            i = get_correct_num(i)
            save_path_final = os.path.join(save_path, f'{i}.jpg')
            if verbose:
                if (int(i)+1)%30==0:
                    print("Extracting frame:", int(i)+1)
                    clear_output(wait=True)
            page.save(save_path_final, 'JPEG')
    except TypeError as e:
        if raise_error:
            raise e
        if verbose:
            print("Extration Done")
    return save_path

In [65]:
file_dir = r"tiff_files\Varying_Height\h1.tif"
extract_frames(file_dir, verbose=True)

Extration Done


c:\Users\harik\anaconda3\envs\project\lib\site-packages\PIL\TiffImagePlugin.py:822: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


'extracted_images\\h1'

### Extracting All The Files

In [6]:
tiff_files = os.listdir(os.path.join("tiff_files", "Varying_Angle"))
tiff_files = tiff_files[1:]
tiff_files

['theta1_bounce.tif',
 'theta1_pass.tif',
 'theta2_bounce.tif',
 'theta2_pass.tif',
 'theta3_bounce.tif',
 'theta3_pass.tif',
 'theta5_bounce1.tif',
 'theta5_bounce2.tif',
 'theta5_pass3.tif',
 'theta5_pass4.tif',
 'theta5_pass5.tif',
 'theta5_pass6.tif',
 'theta6_bounce.tif',
 'theta6_pass.tif']

In [7]:
for file in tqdm.tqdm(tiff_files, desc="Extracting frames"):
    file_dir = r"tiff_files\Varying_Angle\\" + file
    extract_frames(file_dir)

Extracting frames: 100%|██████████| 14/14 [00:30<00:00,  2.18s/it]


## Creating A Text File for each Folder

This text file will be used to store some important information about the images in that sequence of frames. These informations are:
1. Range of frames which are relavent to the image.
   1. First Frame ID (Defualt should be `000.jpg`)
   2. Last Frame ID
2. Frame at which the drop is touching the thin film.
3. Frame at which the drop is leaving the frame.
4. The angle the film makes with horizontal. (Default is 25.017 degrees)

Apart from these, some more information will be added later on.

### The Formate of the Text File

```
### <SEQUENCE_NAME>
### First Frame ID: <FIRST_FRAME_ID>
### Last Frame ID: <LAST_FRAME_ID>
### Cntact Frame ID: <CONTACT_FRAME>
### Leave Frame ID: <LEAVE_FRAME>
### Angle: <ANGLE>
```

Let's create the template.

In [8]:
def get_text(f):
    text = f"""### {f}
### First Frame ID: 
### Last Frame ID: 
### Contact Frame ID: 
### Leave Frame ID: 
### Angle: 
"""
    return text

In [17]:
for f in tqdm.tqdm(tiff_files, desc="Creating Text Files..."):
    text_file_name = f.replace(".tif", ".txt")
    save_path = os.path.join("extracted_images", text_file_name)
    # with open(save_path, "w") as text_file:
    #     text_file.write(get_text(f))

Creating Text Files...: 100%|██████████| 14/14 [00:00<00:00, 823.20it/s]


## Getting Some Information about the sequences

In [10]:
dirs = os.listdir("extracted_images")
dirs = [d for d in dirs if not d.endswith(".txt")]
dirs

['h1',
 'h10_short',
 'h11_short',
 'h12_short',
 'h13_short',
 'h14_short',
 'h15_short',
 'h16_short',
 'h17_short',
 'h18_short',
 'h19_short',
 'h1_try',
 'h2',
 'h20_short',
 'h21_short',
 'h22_short',
 'h23_short',
 'h24',
 'h25_short',
 'h3',
 'h4',
 'h5_short',
 'h6',
 'h7',
 'h7_short',
 'h8_short',
 'h9_short',
 'ref',
 'theta1_bounce',
 'theta1_pass',
 'theta2_bounce',
 'theta2_pass',
 'theta3_bounce',
 'theta3_pass',
 'theta5_bounce1',
 'theta5_bounce2',
 'theta5_pass3',
 'theta5_pass4',
 'theta5_pass5',
 'theta5_pass6',
 'theta6_bounce',
 'theta6_pass']

In [11]:
num_frames = {}
for d in dirs:
    num_frames[d] = len(os.listdir(os.path.join("extracted_images", d)))


In [12]:
df = pd.DataFrame(num_frames, index=["Number of Frames"]).T
df.head()

,Number of Frames
h1,219
h10_short,178
h11_short,166
h12_short,100
h13_short,171


In [13]:
df.to_csv("num_frames.csv")

In [14]:
df.sort_values("Number of Frames", ascending=False).head()

,Number of Frames
h3,370
h2,300
h4,282
h7,273
h1,219


In [15]:
df.sort_values("Number of Frames", ascending=True).head()

,Number of Frames
h25_short,40
theta5_pass5,46
h24,48
h23_short,56
theta3_bounce,67
